In [1]:
import os
import torch
import torch.nn as nn
from torchvision.transforms.functional import resize
from torch.utils.data import DataLoader
import torch.optim as optim
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision import transforms

import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms.functional import pad, resize
from torch.utils.data import DataLoader, Dataset
import numpy as np

torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_peak_memory_stats()

C:\Users\Turog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\cuda\memory.py:343: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [2]:
import torch.nn.functional as F

class SuperResolutionDataset(Dataset):
    def __init__(self, root_dir, patch_size=128, scale_factor=4, max_patches=16):
        self.root_dir = root_dir
        self.patch_size = patch_size
        self.lr_patch_size = patch_size // scale_factor
        self.image_list = os.listdir(root_dir)
        self.scale_factor = scale_factor
        self.max_patches = max_patches  
        self.normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_list[idx])
        image = Image.open(img_path).convert("RGB")
        width, height = image.size

        # Split image into patches
        hr_patches = []
        lr_patches = []
        for y in range(0, height, self.patch_size):
            for x in range(0, width, self.patch_size):
                hr_patch = image.crop((x, y, x + self.patch_size, y + self.patch_size))

                # Skip incomplete patches
                if hr_patch.size[0] != self.patch_size or hr_patch.size[1] != self.patch_size:
                    continue
                lr_patch = resize(hr_patch, (self.lr_patch_size, self.lr_patch_size), interpolation=Image.BILINEAR)
                hr_patches.append(self.normalize(transforms.ToTensor()(hr_patch)))
                lr_patches.append(self.normalize(transforms.ToTensor()(lr_patch)))

        # Pad to max_patches
        while len(hr_patches) < self.max_patches:
            hr_patches.append(torch.zeros((3, self.patch_size, self.patch_size)))
            lr_patches.append(torch.zeros((3, self.lr_patch_size, self.lr_patch_size)))

        hr_patches = hr_patches[:self.max_patches]
        lr_patches = lr_patches[:self.max_patches]

        if len(hr_patches) == 0 or len(lr_patches) == 0:
            raise ValueError(f"No valid patches for image {img_path}")

        return torch.stack(lr_patches), torch.stack(hr_patches)


In [ ]:



from torchvision.models import vgg16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class ResNetSuperResolution(nn.Module):
    def __init__(self, upscale_factor=4):
        super(ResNetSuperResolution, self).__init__()
        
        self.vgg_features = vgg16(pretrained=True).features[:8].eval()  
        for param in self.vgg_features.parameters():
            param.requires_grad = False 

        self.vgg_features.to(device) 

        self.channel_reducer = nn.Conv2d(128, 64, kernel_size=1)  

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)  

        self.conv1 = nn.Conv2d(64, 64, kernel_size=9, padding=4)
        self.relu = nn.ReLU(inplace=True)

        self.residual_blocks = nn.Sequential(*[
            self._make_residual_block(64) for _ in range(5)
        ])
        
        self.upsample1 = nn.Conv2d(64, 64 * (upscale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        
        # Final output layer
        self.conv2 = nn.Conv2d(64, 3, kernel_size=9, padding=4)

    def _make_residual_block(self, channels):
        return nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(channels)
        )

    def forward(self, x):
        with torch.no_grad():  
            x = self.vgg_features(x)

        x = self.channel_reducer(x) 
        x = self.upsample(x)  
        x = self.relu(self.conv1(x))
        residual = x
        x = self.residual_blocks(x)
        x += residual  
        x = self.pixel_shuffle(self.upsample1(x)) 
        x = self.conv2(x) 
        return x


In [5]:
def superres_loss(output, target):
    return F.l1_loss(output, target)


In [6]:
root_dir = r"C:\Users\Turog\OneDrive\Documents\GitHub\576_DL_SuperRes\data\combined_largest_images_rd"
patch_size = 256
scale_factor = 2
dataset = SuperResolutionDataset(root_dir=root_dir, patch_size=patch_size, scale_factor=scale_factor)
dataloader = DataLoader(dataset, batch_size=3, shuffle=False)


In [7]:
upscale_factor = scale_factor
model = ResNetSuperResolution(upscale_factor)
criterion = superres_loss
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
from torch.optim import RMSprop
optimizer = RMSprop(model.parameters(), lr=1e-4, alpha=0.9, weight_decay=1e-5)

os.makedirs("chkt_resnet_v9", exist_ok=True) 
os.makedirs("progress_resnet_v9", exist_ok=True)  


C:\Users\Turog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Turog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    print(f"Memory allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
    print(f"Memory reserved: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

NVIDIA GeForce RTX 4060 Laptop GPU
Memory allocated: 0.00 GB
Memory reserved: 0.00 GB


In [9]:
def pad_image(image, patch_size):
    width, height = image.size
    pad_width = (patch_size - width % patch_size) % patch_size
    pad_height = (patch_size - height % patch_size) % patch_size
    padding = (0, 0, pad_width, pad_height)
    padded_image = pad(image, padding, fill=0)
    return padded_image, padding


def denormalize(tensor):
    denorm = transforms.Normalize(mean=[-1, -1, -1], std=[2, 2, 2])  
    return denorm(tensor)

test_image_path =     r"C:\Users\Turog\OneDrive\Documents\GitHub\576_DL_SuperRes\data\DIV2K\0745.png"
test_image = Image.open(test_image_path).convert("RGB")
test_width, test_height = test_image.size
padded_test_image, padding = pad_image(test_image, patch_size)
padded_width, padded_height = padded_test_image.size


In [11]:
import matplotlib.pyplot as plt
from tqdm import tqdm

normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
def denormalize(tensor):  
    return tensor * 0.5 + 0.5  

os.makedirs("chkt_resnet_v9", exist_ok=True)
os.makedirs("progress_resnet_v9", exist_ok=True)

num_epochs = 20
loss_values = []
lr_patch_size = patch_size // scale_factor

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    if device.type == "cuda":
        print(torch.cuda.get_device_name(0))
        print(f"Memory allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
        print(f"Memory reserved: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")
    
    # Training loop
    for lr_patches, hr_patches in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        if torch.cuda.is_available():
            model = model.cuda()
            lr_patches = lr_patches.cuda()
            hr_patches = hr_patches.cuda()

        lr_patches = lr_patches.view(-1, 3, lr_patch_size, lr_patch_size)
        hr_patches = hr_patches.view(-1, 3, patch_size, patch_size)

        optimizer.zero_grad()
        outputs = model(lr_patches)
        loss = criterion(outputs, hr_patches)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    loss_values.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    torch.save(model.state_dict(), f"chkt_resnet_v9/superres_epoch_{epoch+1}.pth")

    model.eval()
    sr_test_patches = []
    with torch.no_grad():
        for patch in test_patches:
            patch = patch.to(device)
            output = model(patch)
            sr_test_patches.append(denormalize(output).squeeze(0).cpu()) 

    sr_test_image = torch.zeros((3, padded_height, padded_width)) 
    patch_idx = 0
    for y in range(0, padded_height, patch_size):
        for x in range(0, padded_width, patch_size):
            sr_test_image[:, y:y+patch_size, x:x+patch_size] = sr_test_patches[patch_idx]
            patch_idx += 1

    sr_test_image = sr_test_image[:, :test_height, :test_width]

    patch_idx = 0 
    low_res_patch = denormalize(test_patches[patch_idx].squeeze(0).cpu()) 
    super_res_patch = sr_test_patches[patch_idx]
    high_res_patch = denormalize(transforms.ToTensor()(padded_test_image.crop((0, 0, patch_size, patch_size))))  

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.title("Low-Resolution Patch")
    plt.imshow(transforms.ToPILImage()(low_res_patch))
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title(f"Super-Resolved Patch (Epoch {epoch+1})")
    plt.imshow(transforms.ToPILImage()(super_res_patch))
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("High-Resolution Patch")
    plt.imshow(transforms.ToPILImage()(high_res_patch))
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(f"progress_resnet_v9/epoch_{epoch+1}_patch_comparison.png", dpi=300, bbox_inches='tight')
    plt.close()  

    # Plot loss curve
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(loss_values) + 1), loss_values, marker='o', label="Training Loss")
    plt.title("Training Loss Over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    plt.savefig("progress_resnet_v9/loss_plot.png", dpi=200, bbox_inches='tight')
    plt.close()

    plt.figure(figsize=(16, 5))

    plt.subplot(1, 3, 1)
    plt.title(f"Low-Resolution Input ({lr_patch_size}x{lr_patch_size} patches)")
    lr_test_reconstructed = torch.zeros((3, padded_height // scale_factor, padded_width // scale_factor))
    patch_idx = 0
    for y in range(0, padded_height, patch_size):
        for x in range(0, padded_width, patch_size):
            lr_patch = denormalize(test_patches[patch_idx].squeeze(0).cpu())  ### ADDED HERE
            lr_test_reconstructed[:, y // scale_factor:(y + patch_size) // scale_factor,
                                  x // scale_factor:(x + patch_size) // scale_factor] = lr_patch
            patch_idx += 1
    lr_test_reconstructed = lr_test_reconstructed[:, :test_height // scale_factor, :test_width // scale_factor]
    plt.imshow(transforms.ToPILImage()(lr_test_reconstructed))
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title(f"Super-Resolved Output (Epoch {epoch+1})")
    plt.imshow(transforms.ToPILImage()(sr_test_image))
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title(f"Original High-Resolution")
    plt.imshow(test_image)
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(f"progress_resnet_v9/epoch_{epoch+1}_reconstruction_comparison.png", dpi=200, bbox_inches='tight')
    plt.close()

print("Training Complete")


NVIDIA GeForce RTX 4060 Laptop GPU
Memory allocated: 0.00 GB
Memory reserved: 0.00 GB


Epoch 1/20: 100%|██████████| 267/267 [14:43<00:00,  3.31s/it]


Epoch [1/20], Loss: 0.2611
NVIDIA GeForce RTX 4060 Laptop GPU
Memory allocated: 0.06 GB
Memory reserved: 7.09 GB


Epoch 2/20: 100%|██████████| 267/267 [14:05<00:00,  3.17s/it]


Epoch [2/20], Loss: 0.1649
NVIDIA GeForce RTX 4060 Laptop GPU
Memory allocated: 0.06 GB
Memory reserved: 7.10 GB


...

Epoch [18/20], Loss: 0.0848
NVIDIA GeForce RTX 4060 Laptop GPU
Memory allocated: 0.06 GB
Memory reserved: 7.10 GB


Epoch 19/20: 100%|██████████| 267/267 [14:12<00:00,  3.19s/it]


Epoch [19/20], Loss: 0.0834
NVIDIA GeForce RTX 4060 Laptop GPU
Memory allocated: 0.06 GB
Memory reserved: 7.10 GB


Epoch 20/20: 100%|██████████| 267/267 [14:14<00:00,  3.20s/it]


Epoch [20/20], Loss: 0.0824
Training Complete
